In [31]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [32]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [33]:
train_path = 'Dataset/Train'
valid_path = 'Dataset/Test'

In [34]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [35]:
# useful for getting number of classes
folders = glob('Dataset/Train/*')

In [36]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='sigmoid')(x)

In [37]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [38]:
# view the structure of the model
model.summary()


Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [39]:
from keras.preprocessing.image import ImageDataGenerator

In [40]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


In [41]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [42]:
training_set = train_datagen.flow_from_directory('Dataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')

Found 76 images belonging to 2 classes.


In [43]:
test_set = test_datagen.flow_from_directory('Dataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 103 images belonging to 2 classes.


In [56]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [58]:
# fit the model
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=7,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/7
8/8 [==============================] - 41s 5s/step - loss: 0.7033 - accuracy: 0.4342 - val_loss: 0.6935 - val_accuracy: 0.4951
Epoch 2/7
8/8 [==============================] - 43s 5s/step - loss: 0.7022 - accuracy: 0.4737 - val_loss: 0.6908 - val_accuracy: 0.7767
Epoch 3/7
8/8 [==============================] - 43s 5s/step - loss: 0.6978 - accuracy: 0.4079 - val_loss: 0.6868 - val_accuracy: 0.7184
Epoch 4/7
8/8 [==============================] - 43s 5s/step - loss: 0.7643 - accuracy: 0.5000 - val_loss: 0.6768 - val_accuracy: 0.7184
Epoch 5/7
8/8 [==============================] - 45s 6s/step - loss: 0.6991 - accuracy: 0.4737 - val_loss: 0.7014 - val_accuracy: 0.2816
Epoch 6/7
8/8 [==============================] - 43s 5s/step - loss: 0.6929 - accuracy: 0.4474 - val_loss: 0.6723 - val_accuracy: 0.7184
Epoch 7/7
8/8 [==============================] - 43s 5s/step - loss: 0.6925 - accuracy: 0.5263 - val_loss: 0.6927 - val_accuracy: 0.7184


In [76]:

# save model
model.save('final_model.h5')

In [ ]:
# make a prediction for a new image.
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=(224, 224))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, 224, 224, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('sample_image.jpg')
	# load model
	model = load_model('final_model.h5')
	# predict the class
	result = model.predict(img)
	print(result[0])
 
# entry point, run the example
run_example()